"""
This script is used to caclulate the Event Mean Concentration (EMC).
The inputs are .csv files containing concentration and flow after linear interpolation.
"""

In [1]:
import pandas as pd
import numpy as np
from utils.concentration import rainfall_events, emc_cal, conc_interpolate, event_emc
import datetime

# read the discrete storm events
# Read daily loads and flow
# Read hourly loads and flow
from common_settings import obspath, outpath, events_name, \
    obs_events, day_load_flow, hour_load_flow, conct_name, modpath, mod_load_flow

In [2]:
from utils.concentration import cumulative_lq, excel_save
from utils.signatures import update_cumul_df, load_flow_loc

## Produce the event mean concentration of obs and mod

In [3]:
# Calculate EMC for low-frequency data
cols = [col for col in day_load_flow.columns if ('kg' in col) or ('Flow (ML)' in col)]
index_range = [1, obs_events.shape[0]+1]
# index_range2 = [38, obs_events.shape[0]+1]

In [4]:
obs_events = event_emc(obs_events, day_load_flow, index_range, cols[0], cols[1], 
    time_scale='d', multiplier=1)

In [5]:
obs_events.to_csv(outpath + events_name, index='ID')

### Calculate mod load

In [6]:
# read the discrete storm events
filename = 'DIN_mod_storm_event_common3.csv'
events = rainfall_events(f'{outpath}{filename}')
# Calculate EMC for modeling data
cols = [col for col in mod_load_flow.columns if ('Load' in col) or ('ML' in col)]
index_range = [1, events.shape[0]+1]
loads_col = cols[0]; flow_col = cols[1]

events = event_emc(events, mod_load_flow, index_range, loads_col, flow_col, 
    time_scale='d', multiplier=1)
# events.dropna(axis=0, inplace=True)
events.to_csv(f'{outpath}{filename}', index='ID')

## Produce the Normalized cumulative ratio of loads and flow 

### calculate the daily data for double mass plot (Q-L)

In [34]:
time_ranges = [[f'{year}/7/1', f'{year+1}/6/30'] for year in range(2009, 2020)]
# time_ranges = obs_events.loc[:, ['start', 'end']].values
double_mass_ratio = {}
annual_total = pd.DataFrame(columns=day_load_flow.columns)

In [35]:
for ii in range(0, len(time_ranges)-2):
# for ii in range(index_range1[0]-1, index_range1[1]-1):
    df_temp = load_flow_loc(time_ranges[ii], day_load_flow, timestep='d')
    df_temp = update_cumul_df(df_temp, df_temp.values[:, 0], df_temp.values[:, -2])
    double_mass_ratio[f'obs_year_{ii}'] = df_temp
    annual_total.loc[time_ranges[ii][0][0:4]] = df_temp.sum(axis=0)
annual_total.loc['ave'] = annual_total.mean(axis=0)

In [37]:
# save outputs into one excel
annual_total.to_csv(outpath+'obs_annual_sum.csv')
fn = outpath +'obs_year_cumulative_ratio_day.xlsx'
excel_save(double_mass_ratio, fn, True)

### calculate the hourly data for double mass plot (Q-L)

In [15]:
double_mass_ratio = {}
for ii in range(index_range2[0]-1, index_range2[1]-1):
# for ii in range(9, len(time_ranges)):
    df_temp = load_flow_loc(time_ranges[ii], hour_load_flow, timestep='h')
    df_temp = update_cumul_df(df_temp, df_temp.values[:, -1], df_temp.values[:, 0])
    double_mass_ratio[f'obs_storm_{ii}'] = df_temp

In [16]:
# save outputs into one excel
fn = outpath +'obs_storm_cumulative_ratio_hour.xlsx'
excel_save(double_mass_ratio, fn)

### calculate the modeling data for double mass plot (Q-L)

In [39]:
modpath = '../data/mod/'
annual_total = pd.DataFrame(columns=mod_load_flow.columns)
# filename = 'storm_event.csv'
# mod_events = rainfall_events(f'{modpath}{filename}')

In [40]:
# Calculate EMC for modeling data
cols = [col for col in mod_load_flow.columns if ('Load' in col) or ('ML' in col)]
# index_range = [1, mod_events.shape[0]]
loads_col = cols[0]; flow_col = cols[1]

In [41]:
double_mass_ratio = {}
time_ranges = [[f'{year}-07-01', f'{year+1}-06-30'] for year in range(2009, 2018)]
# time_ranges = mod_events.loc[:, ['start', 'end']].values
# for ii in range(index_range[0], index_range[1]):
for ii in range(len(time_ranges)):
    df_temp = load_flow_loc(time_ranges[ii], mod_load_flow, timestep='d')
    df_temp = update_cumul_df(df_temp, df_temp.values[:, 0], df_temp.values[:, -1])
    double_mass_ratio[f'mod_storm_{ii}'] = df_temp
    annual_total.loc[time_ranges[ii][0][0:4]] = df_temp.sum(axis=0)
annual_total.loc['ave'] = annual_total.mean(axis=0)

In [42]:
# save results
annual_total.to_csv(outpath+'mod_annual_sum.csv')

fn = outpath +'mod_year_cumulative_ratio_day.xlsx'
excel_save(double_mass_ratio, fn)

## Calculate event load coefficients

### Event loads for obs

In [7]:
obs_event_fn = 'obs_storm_event_common3'
obs_events = pd.read_csv(f'{outpath}{obs_event_fn}.csv', index_col = 'ID') 
time_ranges = [[f'{year}/7/1', f'{year+1}/6/30'] for year in np.arange(2009, 2018)]
year_loads = {}

In [8]:
# for each year, calculate the yearly loads 
# obs daily data
for tt in time_ranges[0:]:
    df = load_flow_loc(tt, day_load_flow, timestep='d')
    year_loads[tt[0][0:4]] = np.round(df.values[:, 0].sum(), 2)  
    
# obs hourly data
# for tt in time_ranges[-2:]:
#     df = load_flow_loc(tt, hour_load_flow, timestep='h')
#     year_loads[tt[0][0:4]] = np.round(df.values[:, 0].sum(), 2)  

In [9]:
# The event load coefficients
for ii in range(1, index_range[1]):
    df_event = load_flow_loc(obs_events.loc[ii, 'start':'end'].values, day_load_flow, timestep='d')
    ymd= pd.to_datetime(obs_events.loc[ii, 'start'])
    month = ymd.month; year = ymd.year
    if month < 7:
        obs_events.loc[ii, 'event_load_coefficients'] = df_event.values[:, 0].sum() / year_loads[str(year - 1)]
    else:
        obs_events.loc[ii, 'event_load_coefficients'] = df_event.values[:, 0].sum() / year_loads[str(year)]
        
# for ii in range(index_range2[0], index_range2[1]):
#     df_event = load_flow_loc(obs_events.loc[ii, 'start':'end'].values, hour_load_flow, timestep='h')
#     ymd= pd.to_datetime(obs_events.loc[ii, 'start'])
#     month = ymd.month; year = ymd.year
#     if month < 7:
#         obs_events.loc[ii, 'event_load_coefficients'] = df_event.values[:, 0].sum() / year_loads[str(year-1)]
#     else:
#         obs_events.loc[ii, 'event_load_coefficients'] = df_event.values[:, 0].sum() / year_loads[str(year)]

In [10]:
obs_events.to_csv(f'{outpath}{obs_event_fn}.csv')

### Event loads for mod

In [11]:
mod_event_fn = 'DIN_mod_storm_event_common3'
mod_events = pd.read_csv(f'{outpath}{mod_event_fn}.csv', index_col = 'ID') 
time_ranges = [[f'{year}/7/1', f'{year+1}/6/30'] for year in range(2009, 2018)]

In [12]:
# for each year, calculate the yearly loads 
mod_loads = {}
# mod daily data
for tt in time_ranges:
    df = load_flow_loc(tt, mod_load_flow, timestep='d')
    mod_loads[tt[0][0:4]] = np.round(df.values[:, 0].sum(), 2)  

In [13]:
# The event load coefficients
for ii in mod_events.index:
    df_event = load_flow_loc(mod_events.loc[ii, 'start':'end'].values, mod_load_flow, timestep='d')
    ymd= pd.to_datetime(mod_events.loc[ii, 'start'])
    month = ymd.month; year = ymd.year
    if month < 7:
        mod_events.loc[ii, 'event_load_coefficients'] = df_event.values[:, 0].sum() / mod_loads[str(year - 1)]
    else:
        mod_events.loc[ii, 'event_load_coefficients'] = df_event.values[:, 0].sum() / mod_loads[str(year)]

In [14]:
mod_events.to_csv(f'{outpath}{mod_event_fn}.csv')

## Calculate the peaktime difference between flow and loads

### Mod results

In [16]:
mod_event_fn = 'DIN_mod_storm_event_common2'
mod_events = pd.read_csv(f'{outpath}{mod_event_fn}.csv', index_col = 'ID') 

In [17]:
# find the peak time of loads
clabel, llabel, qlabel = ['Downstream Flow Concentration (mg.L^-1)', 'Loads (kg)', 'Flow_cumecs (ML.day^-1)']
for ii in mod_events.index:
    df_event = load_flow_loc(mod_events.loc[ii, 'start':'end'].values, mod_load_flow, timestep='d')
    peaktime_load = df_event[df_event.loc[:, llabel]==df_event.loc[:, llabel].max()].index
    peaktime_conc = df_event[df_event.loc[:, clabel]==df_event.loc[:, clabel].max()].index[0]
    
    mod_events.loc[ii, 'peaktime_load'] = peaktime_load
    mod_events.loc[ii, 'peaktime_conc'] = peaktime_conc
    mod_events.loc[ii, 'peakflow'] = df_event.loc[:, qlabel].max()
    mod_events.loc[ii, 'peaktime'] = df_event[df_event.loc[:, qlabel]==df_event.loc[:, qlabel].max()].index
    mod_events.loc[ii, 'peak_flow_time'] = df_event[df_event.loc[:, qlabel]==df_event.loc[:, qlabel].max()].index[0]
    mod_events.loc[ii, 'peak_conc_time'] = df_event[df_event.loc[:, clabel]==df_event.loc[:, clabel].max()].index[0]
    mod_events.loc[ii, 'peak_load_time'] = df_event[df_event.loc[:, llabel]==df_event.loc[:, llabel].max()].index[0]
    
# mod_events.loc[:, 'delta_time'] = mod_events.peaktime_load - mod_events.peaktime

In [18]:
mod_events.loc[:, 'delta_load_flow_time'] = pd.to_datetime(mod_events.peak_load_time) - pd.to_datetime(mod_events.peak_flow_time)
mod_events.loc[:, 'delta_conc_flow_time'] = pd.to_datetime(mod_events.peak_conc_time) - pd.to_datetime(mod_events.peak_flow_time)

In [19]:
mod_events.to_csv(f'{outpath}{mod_event_fn}.csv')

### Obs results

In [20]:
obs_event_fn = 'obs_storm_event_common2'
obs_events = pd.read_csv(f'{outpath}{obs_event_fn}.csv', index_col = 'ID') 

In [21]:
# find the peak time of loads
for ii in obs_events.index:
#     if ii < 38:
    df_event = load_flow_loc(obs_events.loc[ii, 'start':'end'].values, day_load_flow, timestep='d')
    clabel, llabel, qlabel = ['Concentration (mg/L)', 'Linear_Average_Load(t)', 'Flow(ML)']
#     else:
#         clabel, llabel, qlabel = ['126001A-NO3(mg/l)', 'Loads (kg)', 'Flow (ML)']
#         df_event = load_flow_loc(obs_events.loc[ii, 'start':'end'].values, hour_load_flow, timestep='h')
#     peaktime_load = df_event[df_event.loc[:, llabel]==df_event.loc[:, llabel].max()].index[0]
#     peaktime_conc = df_event[df_event.loc[:, clabel]==df_event.loc[:, clabel].max()].index[0]

    obs_events.loc[ii, 'peaktime_load'] = peaktime_load
    obs_events.loc[ii, 'peaktime_conc'] = peaktime_conc
    obs_events.loc[ii, 'peakflow'] = df_event.loc[:, qlabel].max()
    obs_events.loc[ii, 'peak_flow_time'] = df_event[df_event.loc[:, qlabel]==df_event.loc[:, qlabel].max()].index[0]
    obs_events.loc[ii, 'peak_conc_time'] = df_event[df_event.loc[:, clabel]==df_event.loc[:, clabel].max()].index[0]
    obs_events.loc[ii, 'peak_load_time'] = df_event[df_event.loc[:, llabel]==df_event.loc[:, llabel].max()].index[0]
    
obs_events.loc[:, 'delta_load_flow_time'] = pd.to_datetime(obs_events.peak_load_time) - pd.to_datetime(obs_events.peak_flow_time)
obs_events.loc[:, 'delta_conc_flow_time'] = pd.to_datetime(obs_events.peak_conc_time) - pd.to_datetime(obs_events.peak_flow_time)

In [22]:
obs_events.to_csv(f'{outpath}{obs_event_fn}.csv')

## Variability of load-discharge ratio (seasonal average concentration)

### Obs results

In [22]:
time_ranges = [[f'{year}/7/1', f'{year}/10/1', f'{year+1}/1/1', f'{year+1}/4/1', f'{year+1}/7/1'] for year in range(2009, 2018)]
df_ratio = pd.DataFrame(index=[str(year) for year in range(2009, 2018)], columns = [1, 2, 3, 4])

In [23]:
for tt in time_ranges:
    for ii in range(len(tt) -1):
        start = pd.to_datetime(tt[ii])
        end = pd.to_datetime(tt[ii + 1]) - datetime.timedelta(days=1)   
        df = load_flow_loc([start, end], day_load_flow, timestep ='d')
        df_ratio.loc[tt[0][0:4], ii+1] = df.sum(axis=0)[0] / df.sum(axis=0)[2] * 1000

In [24]:
df_ratio.to_csv(f'{outpath}obs_seasonal_concentration.csv')

### Mod results

In [25]:
time_ranges = [[f'{year}/7/1', f'{year}/10/1', f'{year+1}/1/1', f'{year+1}/4/1', f'{year+1}/7/1'] for year in range(2009, 2018)]
df_ratio = pd.DataFrame(index=[str(year) for year in range(2009, 2018)], columns = [1, 2, 3, 4])

In [26]:
for tt in time_ranges:
    for ii in range(len(tt) -1):
        start = pd.to_datetime(tt[ii])
        end = pd.to_datetime(tt[ii + 1]) - datetime.timedelta(days=1)   
        df = load_flow_loc([start, end], mod_load_flow, timestep ='d')
        df_ratio.loc[tt[0][0:4], ii+1] = df.sum(axis=0)[0] / df.sum(axis=0)[2]

In [27]:
df_ratio.to_csv(f'{outpath}mod_seasonal_concentration.csv')

## Monthly loads

In [30]:
# calculate the monthly loads and flow
df_month = pd.DataFrame(columns = ['obs', 'mod'])
for y in range(2009, 2019):
    for m in range(1, 13):
        start = pd.to_datetime(f'{y}/{m}/1')
        if m == 12:
            end = pd.to_datetime(f'{y+1}/1/1') - datetime.timedelta(days=1) 
        else:
            end = pd.to_datetime(f'{y}/{m+1}/1') - datetime.timedelta(days=1)
            
        df_month.loc[f'{y}/{m}', 'obs'] = 1000 * load_flow_loc([start, end], day_load_flow, timestep ='d').sum(axis=0)[0]
        df_month.loc[f'{y}/{m}', 'mod'] = load_flow_loc([start, end], mod_load_flow, timestep ='d').sum(axis=0)[0]    
df_month = df_month[(df_month.obs != 0) & (df_month.loc[:, 'mod'] != 0)]
df_month.index.name = 'Month'

In [31]:
df_month.to_csv(f'{outpath}mod_obs_month.csv')

## Calculate the coefficients of variation for concentrations (CVC) and discharge (CVQ), their ratio (CVC:CVQ)

In [32]:
# define timeperiod
time_ranges = pd.to_datetime(['2009/7/1', '2018/6/30'])
df_cv = pd.DataFrame(columns=['obs', 'mod'], index=['cvc', 'cvq', 'cvl'])
start, end = time_ranges

In [33]:
# read obs time series of flow, loads and concentration
df_obs = load_flow_loc([start, end], day_load_flow, timestep ='d')
cv_all = (df_obs.std(axis=0) / df_obs.mean(axis=0))
cols = df_obs.columns
df_cv.loc[:, 'obs'] = [cv_all[cols[3]], cv_all[cols[2]], cv_all[cols[0]]]

In [34]:
# read mod time series of flow, loads and concentration
df_mod = load_flow_loc([start, end], mod_load_flow, timestep ='d')
cv_all = (df_mod.std(axis=0) / df_mod.mean(axis=0))
cols = df_mod.columns
df_cv.loc[:, 'mod'] = [cv_all[cols[1]], cv_all[cols[2]], cv_all[cols[0]]]

df_cv.loc['cq', :] = df_cv.loc['cvc', :] / df_cv.loc['cvq', :]
df_cv.loc['lq', :] = df_cv.loc['cvl', :] / df_cv.loc['cvq', :]

In [35]:
df_cv.to_csv(f'{outpath}cv_cql.csv')

## Linear regression of C-Q

In [3]:
# import necessary packages
from sklearn.metrics import r2_score
from utils.signatures import residual, nonlinear_fit
from utils.plotting import regression_plot
import lmfit

In [4]:
# define x and y
time_range = [['/7/1', '/10/1'], ['/10/1', '/1/1'], ['/1/1', '/4/1'], ['/4/1', '/7/1']]
cols = mod_load_flow.columns
# day_load_flow.loc[:, cols[0]] = day_load_flow.loc[:, cols[0]]*1000
cols

Index(['Loads (kg)', 'Downstream Flow Concentration (mg/L)',
       'Flow_cumecs (ML)', 'Unnamed: 4', 'Unnamed: 5'],
      dtype='object')

In [10]:
x_dict, y_dict = {}, {}
k = 1
for tt in time_range:
    x, y = np.array([]), np.array([])
    for year in range(2009, 2019):
        start = pd.to_datetime(f'{year}{tt[0]}')
        if tt[1] == '/1/1':
            end = pd.to_datetime(f'{year+1}{tt[1]}') - datetime.timedelta(days=1)
        else:
            end = pd.to_datetime(f'{year}{tt[1]}') - datetime.timedelta(days=1)
        df_temp = load_flow_loc([start, end], mod_load_flow, timestep ='d')
        x = np.append(x, df_temp.values[:, 2])
        y = np.append(y, df_temp.values[:, 0])
    x_dict[f'{k}_x'] = x
    x_dict[f'{k}_y'] = y
    k += 1

In [ ]:
# variables are x and y
coeff_regress = pd.DataFrame(columns = np.arange(1, 5), index=['R2', 'a', 'b', 'c'])
for k in range(1, 5):
    x = x_dict[f'{k}_x']
    y = x_dict[f'{k}_y']
    p = lmfit.Parameters()
    p.add_many(('a', 0.1, True, 0, 10), ('b', 2, True, 0, 2), ('c', 0, False))
    out1, out2, ci, trace = nonlinear_fit(p, residual, x, y, opti_method='differential_evolution')# lmfit, x=x_input, y=y_output,

    # compare coefficient of determination
    para_values = {}
    for param in ['a', 'b', 'c']: 
        para_values[param] = np.round(trace['a'][param][0], 4)
    y_mod = para_values['a'] * x ** para_values['b']+ para_values['c']
    r2 = r2_score(np.log(y), np.log(y_mod))
    abs_bias = np.abs(np.average(y_mod - y))
    rel_bias = abs_bias / np.average(y)
    coeff_regress.loc[:, k] = [r2, para_values['a'], para_values['b'], para_values['c']]

In [181]:
coeff_regress.to_csv(outpath+'mod_cq_regress.csv')

## Calculate the variation of delivery ratio-surface

In [3]:
# read observations, modeled outputs with delivery ratio at 0 and 25 (%)
mod_fl_fn = 'DIN_sources.csv'
mod_drs0 = pd.read_csv(modpath + mod_fl_fn, index_col='Date')
mod_drs0.index = pd.to_datetime(mod_drs0.index, dayfirst=False)

In [22]:
arrays = [['mod_total','mod_total', 'mod_total', 'mod_total', 'surface', 'surface', 'surface', 'surface', 'seepage', 'seepage', 'seepage', 'seepage', 'DWC', 'DWC', 'DWC', 'DWC',
          'dwc_const', 'dwc_const', 'dwc_const', 'dwc_const', 'emc_const', 'emc_const', 'emc_const', 'emc_const', 'obs', 'obs', 'obs', 'obs'],
          ['median', 'min', 'max', 'mean', 'median', 'min', 'max', 'mean', 'median', 'min', 'max', 'mean', 'median', 'min', 'max', 'mean', 
           'median', 'min', 'max', 'mean', 'median', 'min', 'max', 'mean',  'median', 'min', 'max', 'mean']]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples)

In [5]:
df_month = pd.DataFrame(columns=mod_drs0.columns)
# calculate the monthly loads and flow
for y in range(2009, 2019):
    for m in range(1, 13):
        start = pd.to_datetime(f'{y}/{m}/1')
        if m == 12:
#             import pdb; pdb.set_trace()
            end = pd.to_datetime(f'{y+1}/1/1') - datetime.timedelta(days=1) 
        else:
            end = pd.to_datetime(f'{y}/{m+1}/1') - datetime.timedelta(days=1)
#         
        df_month.loc[f'{y}/{m}', :] = load_flow_loc([start, end], mod_drs0, timestep ='d').sum(axis=0)
df_month = df_month[(df_month.loc[:, df_month.columns[0]] != 0)]
df_month.index.name = 'Month'

In [8]:
df_month.to_csv(f'{outpath}month_loads_sources.csv')

In [9]:
for col in df_month.columns:
    df_month[col+'_ratio'] = (df_month['obs'] - df_month['mod_all'] + df_month[col]) / df_month[col]
#     df_month['drs'] = (df_month['obs'] - df_month['mod_all'] + df_month[col]) / df_month[col]

In [12]:
df_month.to_csv(outpath+'obs_mod_month_load_ratio.csv')

In [24]:
x_list = [*np.arange(7, 13), *np.arange(1, 7)]
drs_stats = pd.DataFrame(columns=x_list, index=index)
drs_stats.index.name = 'month'
col_ratio = df_month.columns[0:7]
for col in col_ratio:
    k = list(col_ratio).index(col)
    for i in range(12):
        drs_stats.loc[index[(4 * k) : (4 * k + 4)], x_list[i]] = df_month[col][i::12].median(), df_month[col][i::12].min(), df_month[col][i::12].max(), df_month[col][i::12].mean()
#         obs_mod = (df_month['obs'][i::12].mean() + df_month[df_month.columns[k+1]][i::12].mean() - df_month['mod_all'][i::12].mean())
#         mod_mod0 = df_month[df_month.columns[k+1]][i::12].mean()
#         drs_stats.loc[index[4 * k + 3], x_list[i]] = obs_mod / mod_mod0
# drs_stats.to_csv(outpath+'DeliveryRatioSurface.csv')

In [26]:
drs_stats.to_csv(outpath+'month_sum_loads.csv')

## Calculate monthly flow and loads for plotting hysteresis loops

In [157]:
df_month_obs, df_month_mod = pd.DataFrame(columns=['Load (kg)', 'Flow (ML)']), pd.DataFrame(columns=['Load (kg)', 'Flow (ML)'])
df_month_mean = pd.DataFrame(columns=[['mod', 'mod', 'obs','obs' ], ['Load (kg)', 'Flow (ML)', 'Load (kg)', 'Flow (ML)']])
# calculate the monthly loads and flow
for y in range(2008, 2019):
    for m in range(1, 13):
        start = pd.to_datetime(f'{y}/{m}/1')
        if m == 12:
            end = pd.to_datetime(f'{y+1}/1/1') - datetime.timedelta(days=1) 
        else:
            end = pd.to_datetime(f'{y}/{m+1}/1') - datetime.timedelta(days=1)   
        df_month_obs.loc[f'{y}/{m}', 'Load (kg)'] = load_flow_loc([start, end], day_load_flow, timestep ='d').sum(axis=0)[0]
        df_month_obs.loc[f'{y}/{m}', 'Flow (ML)'] = load_flow_loc([start, end], day_load_flow, timestep ='d').sum(axis=0)[2]
        df_month_mod.loc[f'{y}/{m}', 'Load (kg)'] = load_flow_loc([start, end], mod_load_flow, timestep ='d').sum(axis=0)[0]
        df_month_mod.loc[f'{y}/{m}', 'Flow (ML)'] = load_flow_loc([start, end], mod_load_flow, timestep ='d').sum(axis=0)[2]

# Delete rows with values of zeros
df_month_obs.drop(inplace=True, index=df_month_obs[df_month_obs['Load (kg)'] == 0].index)
df_month_obs.index = pd.to_datetime(df_month_obs.index)

df_month_mod.drop(inplace=True, index=df_month_mod[df_month_mod['Load (kg)'] == 0].index)
df_month_mod.index = pd.to_datetime(df_month_mod.index)

In [ ]:
# Calculate the monthly average loads and flow
for ii in range(7, 13):
    df_month_mean.loc[f'month_{ii}','mod'] = df_month_mod[df_month_mod.index.month == ii].mean().values
    df_month_mean.loc[f'month_{ii}','obs'] = df_month_obs[df_month_obs.index.month == ii].mean().values
for ii in range(1, 7):
    df_month_mean.loc[f'month_{ii}','mod'] = df_month_mod[df_month_mod.index.month == ii].mean().values
    df_month_mean.loc[f'month_{ii}','obs'] = df_month_obs[df_month_obs.index.month == ii].mean().values
df_month_mean.loc[:, 'Month'] = np.append(np.arange(7, 13), np.arange(1, 7))
df_month_mean.index.name = 'index'
df_month_mean.to_csv(outpath + 'monthly_flow_load_mean.csv')